This notebook provides an example of collecting Broadband Funding Map Data based on a specific date. This is for demostration purposes. 

Documentation for the API can be found here: https://us-fcc.app.box.com/v/bdc-public-data-api-spec

In [1]:
import requests
import json
from datetime import date
import zipfile
import io
import os
from os.path import join, dirname
from dotenv import load_dotenv


The first section establishs today's date, then, the BDC package date is used which is relevant to downloading the correct version of the BDC which associates with the BSLF. The generated BDC date variable can also be used to add a 'fcc_rel' column to the concatenated set of BDC files which is then compared to the BSLF. 

In [3]:
dotenv_path = os.path.join(os.path.dirname('__file__'), '.env')
load_dotenv(dotenv_path)
FCC_USERNAME = os.environ.get('FCC_USERNAME')
FCC_BDC_API_KEY = os.environ.get('FCC_BDC_API_KEY')

The BSLF runs on a 6 month cycle, there is a new version every 12-31-YYYY and 6-30-YYYY. The FCC broadband Map data (BDC), is organized to match the correct BSLF. Data is accessed from an API, but can only be accessed using the correct version of the BSLF date. The API only uses a 12-31-YYYY date or 6-30-YYYY date. 

This logic collects BDC data which matches the BLSF. December data is released ~May 10th and June data is released ~November 10th.

Note that historical BDC data can be collected. 

In [ ]:
today = date.today().timetuple()

In [4]:

if today.tm_mon < 5:
    year = today.tm_year - 1
    bdc_date = str(date(year, 6, 30))
elif today.tm_mon == 5 and today.tm_mday >= 10:
    year = today.tm_year - 1
    bdc_date = str(date(year, 12, 31))
elif today.tm_mon == 5 and today.tm_mday < 10:
    year = today.tm_year - 1
    bdc_date = str(date(year, 6, 30))
elif 11 > today.tm_mon > 5:
    year = today.tm_year - 1 
    bdc_date = str(date(year, 12, 31))
elif today.tm_mon == 11 and today.tm_mday < 10: 
    year = today.tm_year - 1 
    bdc_date = str(date(year, 12, 31))
elif today.tm_mon >= 11 and today.tm_mday >= 10:
    year = today.tm_year
    bdc_date = str(date(year, 6, 30))

Retrieves BDC availability data using API. Further documentation can be seen here: https://us-fcc.app.box.com/v/bdc-public-data-api-spec. Note that there is availability and challenge data available via BDC api. For BDC/BSLF processing, availability data is necessary. 

In [ ]:
def check_read_response(bdc_date):
    url = (
    "https://broadbandmap.fcc.gov/api/public/map/downloads/listAvailabilityData/"
    + bdc_date
    + "?category=State&subcategory=Fixed Broadband"
    )
    headers = {
        "username": FCC_USERNAME,
        "hash_value": FCC_BDC_API_KEY,
    }
## hash value refers to your generated API token
## a standardized api and username will need to be used

    response = requests.get(url, headers=headers)
    read_response = json.loads(response.content)
    return read_response

Defines tech codes of data

10 = Copper Wire

40 = Coaxial Cable

50 = Optical Carrier/Fiber to the Premises

71 = Licensed Terrestrial Fixed Wireless

72 = Licensed by Rule Terrestrial Fixed Wireless 

Gets data specific for the state of Maine with these tech codes. 


In [ ]:
def test_read_response(read_response, bdc_date):
    month = bdc_date[5] + bdc_date[6]

    if bool(read_response.get('data')) is False: 
        if month == '12': 
            bdc_date = bdc_date[0:4] + '-06-30'
            
        elif month == '06': 
            year = int(bdc_date[0:4]) - 1
            bdc_date = str(year) + '-12-31'
            
    else: 
        bdc_date = bdc_date

    return bdc_date

In [6]:
tech_codes = ['10', '40', '50', '71', '72']
file_ids = []
for entry in read_response['data']:
    if entry['state_name'] == 'Maine' and entry['technology_code'] in tech_codes: 
        file_ids.append(entry['file_id']) 

Data is retrived in zip format, all BDC files are extracted. 

In [10]:
for index in range(len(file_ids)): 
    download_url = 'https://broadbandmap.fcc.gov/api/public/map/downloads/downloadFile/availability/' + str(file_ids[index])
    download_response = requests.get(download_url, headers=headers)
    bdc_zip = zipfile.ZipFile(io.BytesIO(download_response.content))
    bdc_zip.extractall()
    bdc_zip.close()